## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [5]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


## Getting started with Collections

In the Earth Engine Data Catalog, datasets can be of different types:

- *Features* which are geometric objects with a list of properties. For example, a watershed with some properties such as *name* and *area*, is an `ee.Feature`.
- *Images* which are like features, but may include several bands. For example, the ground elevation given by the USGS [here](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) is an `ee.Image`.
- *Collections* which are groups of features or images. For example, the [Global Administrative Unit Layers](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0) giving administrative boundaries is a `ee.FeatureCollection` and the [MODIS Land Surface Temperature](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1) dataset is an `ee.ImageCollection`.

If you want to know more about different data models, you may want to visit the [Earth Engine User Guide](https://developers.google.com/earth-engine).

In the following sections, we work with the MODIS [land cover (LC)](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1), the MODIS [land surface temperature (LST)](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1) and with the USGS [ground elevation (ELV)](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003), which are `ee.ImageCollections`. The dataset descriptions provide us with all the information we need to import and manipulate these datasets: the availability, the provider, the Earth Engine Snippet, and the available bands associated with images in the collection.

Now, to import the LC, LST and ELV collections, we can copy and paste the Earth Engine Snippets:

In [6]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

new_ds = ee.ImageCollection('MODIS/061/MOD13Q1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

All of these images come in a different resolution, frequency, and possibly projection, ranging from daily images in a 1 km resolution for LST (hence an `ee.ImageCollection` — a collection of several `ee.Images`) to a single image representing data for the year 2000 in a 30 m resolution for the ELV. While we need to have an eye on the frequency, GEE takes care of resolution and projection by resampling and reprojecting all data we are going to work with to a common projection (learn more about [projections in Earth Engine](https://developers.google.com/earth-engine/guides/projections)). We can define the resolution (called scale in GEE) whenever necessary and of course have the option to force no reprojection.

As you can see in the description of the datasets, they include several sets of information stored in several bands. For example, these bands are associated with the LST collection:

- _LST\_Day\_1km_: Daytime Land Surface Temperature
- _Day\_view\_time_: Local time of day observation
- _LST\_Night\_1km_: Nighttime Land Surface Temperature
- etc.

The description page of the collection tells us that the name of the band associated with the daytime LST is _LST\_Day\_1km_ which is in units of Kelvin. In addition, values are ranging from 7,500 to 65,535 with a corrective scale of 0.02.

Then, we have to filter the collection on the period of time we want. We can do that using the `filterDate()` method. We also need to select the bands we want to work with. Therefore, we decide to focus on daytime LST so we select the daytime band _LST\_Day\_1km_ and its associated quality indicator _QC\_Day_ with the `select()` method.

In [7]:
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

# Initial date of interest (inclusive).
new_i_date = '2017-01-01'

# Final date of interest (exclusive).
new_f_date = '2020-01-01'

new_ds = new_ds.select('NDVI', 'EVI').filterDate(new_i_date, new_f_date)

Now, we can either upload existing shape files or define some points with longitude and latitude coordinates where we want to know more about LC, LST and ELV. For this example, let's use two point locations:

- The first one in the urban area of Lyon, France
- The second one, 30 kilometers away from the city center, in a rural area

In [8]:
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

We can easily get information about our region/point of interest using the following methods (to get more information about available methods and required arguments, please visit the API documentation [here](https://developers.google.com/earth-engine/api_docs)):

- `sample()`: samples the image (does NOT work for an `ee.ImageCollection` — we'll talk about sampling an `ee.ImageCollection` later) according to a given geometry and a scale (in meters) of the projection to sample in. It returns an `ee.FeatureCollection`.
- `first()`: returns the first entry of the collection,
- `get()`: to select the appropriate band of your Image/Collection,
- `getInfo()`: evaluates server-side expression graph and transfers result to client.

Then we can query the ground elevation and LST around our point of interest using the following commands. Please be careful when evaluating LST. According to the [dataset description](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A1), the value should be corrected by a factor of 0.02 to get units of Kelvin (do not forget the conversion). To get the mean multi-annual daytime LST, we use the `mean()` collection reduction method on the LST `ee.ImageCollection`. (The following run might take about 15-20 seconds)

In [9]:
scale = 1000  # scale in meters

# Print the elevation near Lyon, France.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

# print(lc)
# print(lc.first().sample(u_poi, scale))

Ground elevation at urban point: 196 m
Average daytime LST at urban point: 23.12 °C
Land cover value at urban point is: 13


In [10]:
scale = 1000  # scale in meters

# Calculate and print the mean value of the LST collection at the point.
new_urban_point = new_ds.first().sample(u_poi, scale).first().get('NDVI').getInfo()
print('Average NDVI at urban point:', new_urban_point)


Average NDVI at urban point: 3067


In [11]:
from IPython.display import Image

new_img = new_ds.mean()

new_img = new_img.select('NDVI')

roi = u_poi.buffer(1e6)

# Create a URL to the styled image for a region around France.
url = new_img.getThumbUrl({
    'min': 0.0,
    'max': 9000.0,
    'dimensions': 512,
    'region': roi,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301'
    ]})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/93d9d2abe0f3d3fb99145229e0698f93-26b6d46e0890a20f9f7a39963670c4fc:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [13]:
import folium

# Define the center of our map.
lat, lon = 45.77, 4.855

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map



In [14]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [15]:

new_img = new_ds.mean()

new_img = new_img.select('NDVI')

roi = u_poi.buffer(1e6)

# Create a URL to the styled image for a region around France.
url = new_img.getThumbUrl({
    'min': 0.0,
    'max': 9000.0,
    'dimensions': 512,
    'region': roi,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301'
    ]})

# Set visualization parameters for land cover.
new_vis_params = {
'min': 0.0,
    'max': 9000.0,
    # 'dimensions': 512,
    # 'region': roi,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301']
}

# Create a map.
lat, lon = 45.77, 4.855
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(new_img, new_vis_params, 'Land Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [16]:
# locationlist = [["45.77", "4.855"], ["45.77", "5.855"], ["45.77", "6.855"]]
import numpy as np
N = 10

# 111km = 1deg
def km_to_deg(km):
  return km/111

def m_to_deg(m):
  return m/1.11*0.00001

step = 250/1.11*0.00001

radius = km_to_deg(1000)

lan, lon = 45.77, 4.855
points = [lan, lon] + (np.random.random((N,2))*2-1.0)*radius

str_points = [[str(point[0]),str(point[1])] for point in points]

print(str_points)


for point in range(0, len(str_points)):
    folium.Marker(str_points[point]).add_to(my_map)

display(my_map)

[['42.421349791122466', '3.497881563673254'], ['51.625661809369696', '-1.983707345984354'], ['39.56577593317476', '-3.4493927202354904'], ['41.57171901527027', '2.214543257578247'], ['53.191926537950955', '11.110037866268273'], ['39.58235965798923', '9.108224010214236'], ['45.289727684618164', '-1.4721234277263333'], ['37.099630261479305', '13.280194740020791'], ['50.02364089589382', '10.560293553939538'], ['51.76689137393488', '10.145143519650158']]
